### Imports and CUDA

In [1]:
# Matplotlib
import requests
import matplotlib.pyplot as plt
# Numpy
import numpy as np
# Torch
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch.nn as nn

import pandas as pd

In [2]:
# Use GPU if available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


### Objective

#### To develop a model that predicts taxi availability within a specific area for the next three hours. This means that if the model is run at 12 PM, it will provide predicted taxi availability for 1 PM, 2 PM, and 3 PM.

The area of interest is defined by the following geographical boundaries:

    North: 1.35106
    South: 1.32206
    East: 103.97839
    West: 103.92805

To identify the taxis currently available within this region, we use the TaxiAvailabilityScript.py.

This script collects real-time data, which serves as input for our predictive model.

By leveraging historical taxi availability trends and real-time data, our model aims to provide accurate forecasts, helping commuters, ride-hailing services, and urban planners make informed decisions.


# **To-Do List for Taxi Availability Prediction**

## **Step 1: Cleaning the Taxi Availability Data**
The first step involves retrieving and preprocessing the taxi availability dataset. The dataset consists of the following columns:

1. **DateTime**  
2. **Taxi Available Throughout Singapore**  
3. **Taxi Available in Selected Box Area**  
4. **Coordinates[]**  

For our specific use case, **the coordinates column will not be used for now**.  

To prepare the data for the neural network:  
- **Inputs:** We will use `DateTime` and `Taxi Available Throughout Singapore` as features.  
- **Output:** `Taxi Available in Selected Box Area` will be the target variable.  
- **DateTime Conversion:** Since `DateTime` is not in a format suitable for neural networks, we will extract relevant features:
  - **IsWeekend**: A binary feature (1 if it's a weekend, 0 otherwise).  
  - **Hour**: Transformed into a numerical value between **1 and 24** (avoiding 0, which may cause training issues).  

---

## **Step 2: Adding Additional Features**  
*(Partially completed; will be refined over time)*  

Aside from the existing columns, we aim to incorporate additional features that may improve prediction accuracy:  

1. **ERP Rates (Electronic Road Pricing) at the given time and location**  
   - Uncertain if this will significantly impact predictions. Further analysis is needed.  

2. **Number of LTA (Land Transport Authority) gantry locations**  
   - Again, its usefulness remains uncertain—further evaluation required.  

3. **Traffic Incidents in the Selected Area**  
   - A script (`TrafficIncidentScript.py`) has been written to update `traffic_incident.csv` with the latest traffic incidents.  
   - Over time, as the dataset grows, we hope this feature will become useful.  

4. **Number of Taxi Stands in the Area**  
   - Currently **not useful** because our area of interest is fixed.  
   - However, if we allow dynamic selection of areas in the future, this could become relevant.  

5. **Temperature at a Given Time and Date** *(To be implemented)*  

6. **Rainfall Data** *(To be implemented)*  

To ensure all features align properly, we will **synchronize all datasets based on DateTime** before feeding them into the model.  

---

## **Step 3: Creating the Training-Test Split**  
- Initially, we will perform an **80/20 Training-Test split** for simplicity.  
- In the future, we may introduce a **Training-Validation-Test split** to further refine model performance.  

---

## **Step 4: Building the Model**  
We will begin with an **LSTM model**, as LSTMs are well-suited for time-series forecasting.  
- **Initial Limitation:** The model, in its basic form, will only predict the next hour.  
- **Future Improvement:** A **sliding window approach** will be explored and implemented to extend predictions further.  

---

## **Step 5: Model Evaluation and Improvement**  
- After the initial model is trained, we will assess its performance.  
- Based on results, we will explore potential improvements, such as hyperparameter tuning, architectural modifications, or additional feature engineering.  

---

This structured approach will guide the development of a robust and accurate taxi availability prediction model. 🚖💡


## **Preparing the taxi_availability data here.**

Normalization of certain inputs are done as well, but I am unsure if it is the right thing to do as well.

In [3]:
# weather_file_path = "merged_weather.csv"
# weather_df = pd.read_csv(weather_file_path, delimiter=",")

# weather_df = weather_df[::-1]


In [4]:
# taxi_availability_file_path = "taxi_availability.csv"

# taxi_df = pd.read_csv(taxi_availability_file_path, delimiter=",")

merged_weather_taxi_df = "merged_file.csv"
taxi_df = pd.read_csv(merged_weather_taxi_df, delimiter = ",")

#Adjusting for weather parameters
taxi_df = taxi_df.drop(columns = "stationId")

#Adjusting for taxi_vailability parameters
taxi_df_coordinates = taxi_df["Coordinates[]"]
taxt_df_datetime = taxi_df["DateTime"]
taxi_df = taxi_df.drop(columns = "Coordinates[]")
taxi_df["DateTime"] = pd.to_datetime(taxi_df["DateTime"])

taxi_df["IsWeekend"] = (taxi_df["DateTime"].dt.weekday >= 5).astype(int)
taxi_df["Hour"] = taxi_df["DateTime"].dt.hour + 1  # Convert 0-23 to 1-24
taxi_df = taxi_df.drop(columns = "DateTime")

print(taxi_df.dtypes)

Taxi Available throughout SG             int64
Taxi Available in Selected Box Area      int64
temp_value                             float64
humidity_value                         float64
rainfall_value                         float64
IsWeekend                                int64
Hour                                     int32
dtype: object


### Converting all dtypes into float32

In [5]:
# taxi_df=taxi_df[:5120]
numeric_columns = taxi_df.select_dtypes(include=['int64', 'int32','float64','object']).columns
print("numeric_columns",numeric_columns)
taxi_df[numeric_columns] = taxi_df[numeric_columns].astype('float32')
numeric_columns = taxi_df.select_dtypes(include=['int64', 'int32','float64','object']).columns

# Convert selected columns to float32
taxi_df[numeric_columns] = taxi_df[numeric_columns].astype('float32')




numeric_columns Index(['Taxi Available throughout SG', 'Taxi Available in Selected Box Area',
       'temp_value', 'humidity_value', 'rainfall_value', 'IsWeekend', 'Hour'],
      dtype='object')


### Normalizing all values

In [6]:
#---------------Normalise-----------------------
# Drop 'DateTime' as it's no longer needed
# Normalize the 'Hour' and 'IsWeekend' columns (if needed)


data_min = taxi_df.min(axis=0)
data_max = taxi_df.max(axis=0)
taxi_df_normalized = (taxi_df - data_min) / (data_max - data_min)

# scaler = MinMaxScaler()
# taxi_df[["Hour", "IsWeekend", "Taxi Available throughout SG","Taxi Available in Selected Box Area"]] = scaler.fit_transform(taxi_df[["Hour", "IsWeekend","Taxi Available throughout SG","Taxi Available in Selected Box Area"]])
# taxi_df = taxi_df.apply(pd.to_numeric, errors='coerce')
taxi_df_output_normalized  = taxi_df_normalized["Taxi Available in Selected Box Area"]
taxi_df_normalized = taxi_df_normalized.drop(columns = "Taxi Available in Selected Box Area")
taxi_df_normalized.to_csv("checker.csv", index=False)  # Set index=False to exclude row numbers

# Convert to NumPy arrays
input_data = taxi_df_normalized.values  # Shape: (5120, num_features)
output_data = taxi_df_output_normalized.values  # Shape: (5120,)
# taxi_df_output = taxi_df["Taxi Available in Selected Box Area"]
# input_data = taxi_df.values  # Shape: (5120, num_features)
# output_data = taxi_df_output.values  # Shape: (5120,)

print("Input Data: ",input_data.shape)
print("Output Data: ",output_data.shape)

# Define sequence length



Input Data:  (8482, 6)
Output Data:  (8482,)


### No Normalization Style

In [7]:
# #---------------No Normalization-----------------------

# # Drop 'DateTime' as it's no longer needed
# taxi_df_no_norm = taxi_df  # Remove DateTime but keep raw values

# # Separate input and output data
# taxi_df_output_no_norm = taxi_df_no_norm["Taxi Available in Selected Box Area"]
# taxi_df_no_norm = taxi_df_no_norm.drop(columns=["Taxi Available in Selected Box Area"])

# # Save to CSV for checking
# taxi_df_no_norm.to_csv("checker_no_norm.csv", index=False)  # Set index=False to exclude row numbers

# # Convert to NumPy arrays (raw values)
# input_data = taxi_df_no_norm.values  # Shape: (5120, num_features)
# output_data = taxi_df_output_no_norm.values  # Shape: (5120,)

# print("Input Data: ", input_data.shape)
# print("Input Data: ", input_data[0])

# print("Output Data: ", output_data.shape)
# print("Input Data: ", output_data[0])


### Create Sequence Function

In [8]:
seq_length = 24
pred_horizon = 3  # Number of future time steps to predict

def create_sequences(data, labels, seq_length, pred_horizon):
    xs, ys = [], []
    for i in range(0, len(data), seq_length):  # Start from 0 and increment by seq_length
        if i + seq_length + pred_horizon <= len(data):  # Ensure enough data for prediction horizon
            xs.append(data[i:i + seq_length])  # Input sequence (continuous)
            ys.append(labels[i + seq_length : i + seq_length + pred_horizon])  # Next 3 values
    return np.array(xs), np.array(ys)


In [9]:

X, y = create_sequences(input_data, output_data, seq_length,pred_horizon)


# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)  # Add channel dim
y = torch.tensor(y[:, None], dtype=torch.float32)

# Split into training (80%) and testing (20%)
train_size = int(0.8 * len(X))
test_size = len(X) - train_size

trainX, testX = X[:train_size], X[train_size:]
trainY, testY = y[:train_size], y[train_size:]

# Convert to PyTorch tensors
trainX = torch.tensor(trainX, dtype=torch.float32)  # Shape: (train_size, 24, num_features)
trainY = torch.tensor(trainY, dtype=torch.float32)  # Shape: (train_size,)
testX = torch.tensor(testX, dtype=torch.float32)    # Shape: (test_size, 24, num_features)
testY = torch.tensor(testY, dtype=torch.float32)    # Shape: (test_size,)

train_dataset = TensorDataset(trainX, trainY)
test_dataset = TensorDataset(testX, testY)

batch_sizes = 17  # You can adjust the batch size as needed
#Shuffle is false because you need it to be in time sequenced, but please double check
train_loader = DataLoader(train_dataset, batch_size=batch_sizes, shuffle=False,drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_sizes, shuffle=False, drop_last=True)

# Example of accessing a batch of data
for inputs, targets in train_loader:
    print(f'Inputs: {inputs.shape}, Targets: {targets.shape}')
    break  # Only print the first batch for verification

Inputs: torch.Size([17, 24, 6]), Targets: torch.Size([17, 1, 3])


C:\Users\nicho\AppData\Local\Temp\ipykernel_10412\3437087755.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  trainX = torch.tensor(trainX, dtype=torch.float32)  # Shape: (train_size, 24, num_features)
C:\Users\nicho\AppData\Local\Temp\ipykernel_10412\3437087755.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  trainY = torch.tensor(trainY, dtype=torch.float32)  # Shape: (train_size,)
C:\Users\nicho\AppData\Local\Temp\ipykernel_10412\3437087755.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  testX = torch.tensor(testX, dtype=t

In [10]:
print("test_loader",len(test_loader))

test_loader 4


Using a bidirectional LSTM as the encoder

In [11]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers = 1, dropout = 0.0):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers = num_layers, 
                           dropout = dropout if num_layers > 1 else 0.0, 
                           batch_first = True)

    def forward(self, input_seq):
        # For this encoder, we ignore the outputs if we only need the final hidden state(s)
        outputs, hidden = self.rnn(input_seq)
        return hidden

In [12]:
class DecoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers = 1, dropout = 0.0):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers=num_layers, 
                           dropout=dropout if num_layers > 1 else 0.0, 
                           batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)  # Output linear layer
        self.project = nn.Linear(output_size, input_size)  # Projection layer to get input_size for next time step

    def forward(self, input_seq, hidden):
        # Get the raw output from the RNN along with updated hidden state(s)
        output, hidden = self.rnn(input_seq, hidden)
        output = self.linear(output)  # Decode to scalar output
        output = self.project(output)  # Project to input_size for the next time step
        return output, hidden


In [13]:
class Seq2Seq(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers = 1, dropout = 0.0):
        super(Seq2Seq, self).__init__()
        self.num_layers = num_layers
        self.encoder = EncoderRNN(input_size, hidden_size, num_layers, dropout)
        self.decoder = DecoderRNN(input_size, hidden_size, output_size, num_layers, dropout)

    def forward(self, input_seq, target_seq_length):
        # Encoder part: obtain the hidden state from the encoder.
        encoder_hidden = self.encoder(input_seq)
        
        # Prepare a non-autoregressive decoder input.
        decoder_input = input_seq[:, -1].unsqueeze(1)  # (batch_size, 1, input_size)

        # Generate the outputs in an autoregressive manner via a for loop.
        outputs = []
        hidden = encoder_hidden
        for _ in range(target_seq_length):
            # Unpack the tuple returned by the decoder: (output, updated_hidden)
            decoder_output, hidden = self.decoder(decoder_input, hidden)
            outputs.append(decoder_output)

            # Use the decoder's output as the next input, which is projected to input_size
            decoder_input = decoder_output

        outputs = torch.cat(outputs, dim=1)  # (batch_size, target_seq_length, output_size)
        return outputs


In [14]:
def train_seq2seq(model, dataloader, num_epochs, learning_rate, device, future_steps):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()  # For regression (single scalar output)

    for epoch in range(num_epochs):
        epoch_loss = 0
        for batch in dataloader:
            inputs, targets = batch  # inputs shape: (batch_size, seq_len, input_size)
            inputs = inputs.to(device)
            targets = targets.to(device)

        

            # Forward pass through the model
            output = model(inputs, future_steps)  # Predict the scalar output

            # Calculate loss (output shape is (batch_size, 1))
            batch_loss = criterion(output, targets)
            epoch_loss += batch_loss.item()

            # Backward pass and optimization
            optimizer.zero_grad()
            batch_loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(dataloader)}")

    return epoch_loss


In [15]:
# Hyperparameters
input_size = 6
hidden_size = 100
output_size = 6
learning_rate = 0.001
num_epochs = 100
num_layers = 3
dropout_rate = 0.1
dataloader = train_loader

# Initialize Seq2Seq Model
seq2seq_model = Seq2Seq(input_size, hidden_size, output_size, num_layers, dropout_rate)

# Train the model
loss_graph = train_seq2seq(seq2seq_model, dataloader, num_epochs, learning_rate, device, future_steps=3)
# Plot the loss graph
plt.plot(loss_graph)
plt.title("Loss Graph")
plt.xlabel("Epochs")
plt.ylabel("Loss")


c:\Users\nicho\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([17, 1, 3])) that is different to the input size (torch.Size([17, 3, 6])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (6) must match the size of tensor b (3) at non-singleton dimension 2

In [ ]:
print(len(test_loader))

In [ ]:
model.eval()

# Initialize variables to track loss
loss_value = 0
num_batches = 0

# Define the loss function
criterion = torch.nn.MSELoss()

# Initialize hidden state and cell state
hidden_state, cell_state = None, None  

# Disable gradient computation for validation
with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        if batch_idx == len(test_loader) - 1:  
            break  # Skip the last batch

        # Forward pass
        output, cell_state, hidden_state = model(inputs, cell_state, hidden_state)

        # Denormalize predictions and targets (for all 3 time steps)
        output_denorm = output * (data_max["Taxi Available in Selected Box Area"] - data_min["Taxi Available in Selected Box Area"]) + data_min["Taxi Available in Selected Box Area"]
        targets_denorm = targets * (data_max["Taxi Available in Selected Box Area"] - data_min["Taxi Available in Selected Box Area"]) + data_min["Taxi Available in Selected Box Area"]

        # Compute loss on normalized data
        loss_value += criterion(output, targets)

        # Print a sample of the normalized and denormalized values
        print("Normalized output[0]:", output[0].tolist())  
        print("Normalized target[0]:", targets[0].tolist())  
        print("Denormalized output[0]:", output_denorm[0].tolist())  
        print("Denormalized target[0]:", targets_denorm[0].tolist())  
        print("-" * 50)

# Compute average loss
loss_value = loss_value / (len(test_loader) - 1)
print("Predicted output shape:", output.shape)
print("True output shape:", targets.shape)
print(f'Average Validation Loss: {loss_value:.4f}')


In [ ]:

# model.eval()

# # Initialize variables to track loss
# loss_value = 0
# num_batches = 0

# # Define the loss function
# criterion = torch.nn.MSELoss()

# # Initialize hidden state and cell state
# hidden_state, cell_state = None, None  

# # Disable gradient computation for validation
# with torch.no_grad():
#     for batch_idx, (inputs, targets) in enumerate(test_loader):
#         if batch_idx == len(test_loader) - 1:  
#             break  # Skip the last batch
#         # Forward pass
#         output, cell_state, hidden_state = model(inputs, cell_state, hidden_state)
#         print("Predicted output: ", output)
#         print("True Output: ", targets)

#         # Compute loss
#         # Compute loss
#         loss_value += criterion(output, targets)
# # Compute average loss
# loss_value = loss_value / (len(test_loader) -1 )

# # Print validation results
# print(f'Average Validation Loss: {loss_value:.4f}')
